In [25]:
import networkx as nx   
import axelrod as axl  

In [26]:
import numpy as np
import pandas as pd
import random
#import statsmodels.formula.api as sm

In [17]:
# The current winning strategies of a round robin tournament in axelrod library.
players = [axl.Gambler(), axl.EvolvedLookerUp(), axl.DoubleCrosser(), axl.FoolMeOnce(),
           axl.Gradual(), axl.BackStabber(), axl.ForgetfulFoolMeOnce(), axl.MetaHunter(),
           axl.OmegaTFT(), axl.MetaMajorityLongMemory(), axl.MetaMajorityFiniteMemory(),
           axl.Grudger(), axl.MetaMajorityMemoryOne(), axl.MetaWinner(), axl.NiceAverageCopier(),
           axl.Shubik(), axl.ZDGTFT2(), axl.SoftGrudger(), axl.SoftJoss(), axl.LimitedRetaliate2()]

In [27]:
# ordinary strategies of axelrod
ordinary_players = [s() for s in axl.ordinary_strategies]

In [5]:
# random strategies of the ordinary strategies
random_players = random.sample(ordinary_players, 50)

In [29]:
def Neighbors(G):
    """
    Returns a list with the neighbors of a player.
    
    By neighbors we mean the players a player
    interacts with.
    """
    Neighbors = []
    for i in range(len(players)) : Neighbors.append(G.neighbors(i))
    return Neighbors

In [30]:
def Neighborhood_size (Neighborhood) :
    """
    Returns the size of the neighborhood of a
    player.
    """
    Neighborhood_size = []
    for i in range(len(Neighborhood)) : Neighborhood_size.append(len(Neighborhood[i]))
    return Neighborhood_size

In [31]:
def Neighbors_Scores(G, results):
    """
    Returns a list with the scores of the 
    neighbors.
    """
    return [[results.scores[j] for j in G.neighbors(i)]
            for i, pl in enumerate(players)]

In [32]:
def Average_Neighborhood_Score(G, results) :
    """
    Returns the average score of the
    sum of the neighbors scores.
    
    Thus, the average score of the neighborhood.    
    """
    av_score = []
    temp = [[j for m in G.neighbors(k) for j in results.scores[m]]
              for k in G.nodes()]
    for i in G.nodes() :
        av_score.append(np.mean(temp[i]))
    return av_score

In [33]:
def Cliques(G) :
    """
    Return a list of cliques the player belongs to
    """
    nx.find_cliques(G)
    cliques = []
    for i in G.nodes() : 
        cliques.append(nx.cliques_containing_node(G, i))
    return cliques    

In [34]:
def touranment(G, p, seed):
    """
    Run a spatial tournament with a topology
    of any given graph. Return the results.
    """
    # set seed the tournament. 
    random.seed(seed)
    np.random.seed(seed)
    edges = G.edges()
    
    # create tournament
    tournament = axl.SpatialTournament(players, edges=G.edges(), turns = 200, repetitions=5)
    
    # play the tournament. Return the results.
    return tournament.play(processes=0,
                           filename="/home/nikoleta/src/Dissertation-Notebooks/Data/Cycle/Cycle-{}.csv".format(seed))

In [35]:
def tournament_results(G, seed,p):
    """
    Creates a data frame with parameters of the tournament.
    
    Parameters
    ----------
    players_list : list
            A list with the players participating in the tournament
    seed : integer
            A seed for the tournament and graph
    player_index : integer
            A players index number
    player_name : character
            The name of the player           
    degree : integer
            THe degree of the graph
    neighbors : list
            A list with the player's neighbors index number
    neighborhood_size: integer
            The size of the neighborhood, should be eqaul with the degree
   
    """
    # generate the tournament and the results
    results = touranment(G, p, seed)
    
    # parameters 
    neighborhood = Neighbors(G)
    nsize = Neighborhood_size(neighborhood)
    degree = list(G.degree(G.nodes()).values())
    nscores = Neighbors_Scores(G, results)
    av_nscores = Average_Neighborhood_Score(G, results)
    cliques = Cliques(G)
    
    
    # create data frame
    data = pd.DataFrame({'players_list' : [players for _ in results.players],
                         'seed' : seed, 'parameter': p ,'player_index' : G.nodes(),
                         'player_name' : results.players, 'degree' : degree ,
                         'neighbors' : neighborhood, 'neighborhood_size' : nsize,
                         'ranking' : results.ranking, 'scores' : results.scores,
                         'normalised_scores' : results.normalised_scores,
                         'average_score' : [np.median(scores) for scores in results.normalised_scores],
                         'neighbors_scores' : nscores, 'average_neighboorhood_score' : av_nscores,
                         'R' : [list(results.game.RPST())[0] for _ in results.players],
                         'P' : [list(results.game.RPST())[1] for _ in results.players],
                         'S' : [list(results.game.RPST())[2] for _ in results.players],
                         'T' : [list(results.game.RPST())[3] for _ in results.players],
                         'connectivity' : nx.node_connectivity(G), 
                         'clustering' : nx.average_clustering(G), 
                         'cliques' : cliques
                        })
   
    return data

In [36]:
# define the players


In [37]:
cols = ['players_list', 'seed', 'parameter', 'player_name', 'player_index', 
        'degree', 'neighbors', 'neighborhood_size', 'ranking', 'scores',
        'normalised_scores',  'average_score', 'neighbors_scores',
        'average_neighboorhood_score', 'R', 'P', 'S', 'T', 
        'clustering', 'connectivity', 'cliques']

In [12]:
# for the binomial graphs
"""
nodes = False
while nodes != list(range(len(players))): 
    G = nx.binomial_graph(len(players), p=.1 )
    nodes = list(set([node for edge in G.edges() for node in edge]))
"""

In [38]:
results = pd.DataFrame()
for seed in range(0, 100):
    
    np.random.seed(seed)
    p = seed/100
    
    # define the graph  
    players = random.sample(ordinary_players, 100)
    G = nx.cycle_graph(len(players))
    
    results = results.append([tournament_results(G, seed,p)])

results = results[cols]
results.to_csv("/home/nikoleta/src/Dissertation-Notebooks/Data/Cycle/Cycle.csv", index=False) 


In [56]:
results

,players_list,seed,parameter,player_name,player_index,degree,neighbors,neighborhood_size,ranking,scores,...,average_score,neighbors_scores,average_neighboorhood_score,R,P,S,T,clustering,connectivity,cliques
0,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,Adaptive,0,2,"[1, 131]",2,19,"[586, 628, 616, 606, 597]",...,1.5150,"[[823, 417, 823, 823, 823], [901, 880, 867, 32...",699.5,3,1,0,5,0.0,2,"[[0, 1], [0, 131]]"
1,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,Aggravater,1,2,"[0, 2]",2,124,"[823, 417, 823, 823, 823]",...,2.0575,"[[586, 628, 616, 606, 597], [1191, 800, 891, 8...",799.7,3,1,0,5,0.0,2,"[[0, 1], [2, 1]]"
2,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,ALLCorALLD,2,2,"[1, 3]",2,35,"[1191, 800, 891, 891, 1191]",...,2.2275,"[[823, 417, 823, 823, 823], [221, 221, 921, 92...",621.4,3,1,0,5,0.0,2,"[[2, 1], [2, 3]]"
3,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,Alternator,3,2,"[2, 4]",2,12,"[221, 221, 921, 921, 221]",...,0.5525,"[[1191, 800, 891, 891, 1191], [1137, 1137, 113...",1064.9,3,1,0,5,0.0,2,"[[2, 3], [3, 4]]"
4,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,Alternator Hunter,4,2,"[3, 5]",2,121,"[1137, 1137, 1137, 1137, 1137]",...,2.8425,"[[221, 221, 921, 921, 221], [711, 711, 711, 71...",606.0,3,1,0,5,0.0,2,"[[3, 4], [4, 5]]"
5,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,AntiCycler,5,2,"[4, 6]",2,9,"[711, 711, 711, 711, 711]",...,1.7775,"[[1137, 1137, 1137, 1137, 1137], [1321, 1321, ...",1229.0,3,1,0,5,0.0,2,"[[4, 5], [5, 6]]"
6,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,Anti Tit For Tat,6,2,"[5, 7]",2,56,"[1321, 1321, 1321, 1321, 1321]",...,3.3025,"[[711, 711, 711, 711, 711], [1076, 1076, 1076,...",893.5,3,1,0,5,0.0,2,"[[5, 6], [6, 7]]"
7,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,Adapative Pavlov 2006,7,2,"[8, 6]",2,109,"[1076, 1076, 1076, 1076, 1076]",...,2.6900,"[[1200, 1200, 1200, 1200, 1200], [1321, 1321, ...",1260.5,3,1,0,5,0.0,2,"[[6, 7], [7, 8]]"
8,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,Adapative Pavlov 2011,8,2,"[9, 7]",2,113,"[1200, 1200, 1200, 1200, 1200]",...,3.0000,"[[1405, 1474, 1513, 1329, 1352], [1076, 1076, ...",1245.3,3,1,0,5,0.0,2,"[[7, 8], [8, 9]]"
9,"[Adaptive, Aggravater, ALLCorALLD, Alternator,...",0,0.00,Appeaser,9,2,"[8, 10]",2,6,"[1405, 1474, 1513, 1329, 1352]",...,3.5125,"[[1200, 1200, 1200, 1200, 1200], [439, 355, 25...",816.3,3,1,0,5,0.0,2,"[[8, 9], [9, 10]]"
